In [63]:
import os
from dotenv import load_dotenv

load_dotenv()

model_name = "openai:gpt-4o-mini"


In [59]:
system_prompt = """
You are an expert in finding Russian-language Telegram channels.
Your primary task is to find up to 10 existing and active channels related to a given topic,
and return them with some description.

Key Requirements:
1. Channel Verification:
   - Verify the existence of each channel before including it
   - Only include channels you have personally verified
   - Ensure channels are currently active
   - Only include channels with proven track record of reliability and trustworthiness
   - Prioritize channels with established reputation and credibility

2. Search Requirements:
   - Focus on finding the most popular and relevant channels
   - Prioritize channels with high subscriber counts and regular activity
   - Ensure channels have a history of providing accurate and reliable information


Response Format:
- Up to 10 channel usernames
- Urls are in @channelname format
- Description is a brief description of the channel's content and focus area

CHECK YOUR ANSWER BEFORE RETURNING IT!!!
"""

In [60]:
from pydantic import BaseModel, Field


class SourceGenerateResponse(BaseModel):
    url: str = Field(
        ...,
        description="The URL of the Telegram channel in @channelname format",
        examples=["@python_ru", "@devops_ru"]
    )
    description: str = Field(
        ...,
        description="Brief description of the channel's content and focus area",
        min_length=10,
        max_length=200,
        examples=["Channel where you can find actual news about Python programming and development"]
    )

In [61]:
from typing import List
from pydantic_ai import Agent
from pydantic_ai.common_tools.duckduckgo import duckduckgo_search_tool


tools = [duckduckgo_search_tool()]
agent = Agent(
    model=model_name,
    tools=tools,
    deps_type=str,
    output_type=List[SourceGenerateResponse],
    system_prompt=system_prompt,
    retries=10
)

In [62]:


result = await agent.run("Hacking")
print(result)
print(result.output)
for channel in result.output:
    print(f"""
    URL: {channel.url}
    DESCRIPTION: {channel.description}

    """)



AgentRunResult(output=[SourceGenerateResponse(url='@thehackernews', description='Official THN Telegram Channel, a trusted source for breaking news and tech coverage about cybersecurity and hacking.'), SourceGenerateResponse(url='@universityofkalilinux', description='Channel dedicated to learning offensive security and hacking using Kali Linux.'), SourceGenerateResponse(url='@HackersFeed', description='A comprehensive channel providing hacking news from over 100 sources, including exploits, new CVEs, and pentesting tutorials.'), SourceGenerateResponse(url='@cybersecuritychannel', description='Focuses on sharing information about cybersecurity threats, vulnerabilities, and protective measures.'), SourceGenerateResponse(url='@darkweb_now', description='Explores dark web activities including illicit markets, cybersecurity breaches, and hacker communities.'), SourceGenerateResponse(url='@hackingworld', description='Updates on global hacking news, tools, vulnerabilities, and community discus